In [ ]:
# Import necessary packages

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from scipy.stats import norm
from operator import itemgetter
from sklearn.model_selection import cross_val_score

In [ ]:
# Import the datasets

dfHistorical = pd.read_csv('historical-mvps.csv')
dfCurrent = pd.read_csv('current-mvp-ladder.csv')
dfChange = pd.read_csv('predictions-change.csv')

In [ ]:
# Preview the historical dataset

dfHistorical.head()

# Create models

In [ ]:
train, test = train_test_split(dfHistorical, test_size = 0.25, random_state = 10)

xtrain = train[['Team Wins', 'Overall Seed', 'PTS', 'TRB', 'AST', 'FG%', 'VORP', 'WS']]
ytrain = train[['Share']]

xtest = test[['Team Wins', 'Overall Seed', 'PTS', 'TRB', 'AST', 'FG%', 'VORP', 'WS']]
ytest = test[['Share']]

In [ ]:
# Create function that gives accuracy scores for each model

def scores(y, model):
    
    model.fit(xtrain, ytrain.values.ravel())
    y_pred = model.predict(xtest)
    
    print("Mean squared error: %.3f" % mean_squared_error(ytest, y_pred))
    print('R2 score: %.3f' % r2_score(ytest, y_pred))

    cvScore = cross_val_score(model, xtest, ytest.values.ravel(), cv = 3, scoring = 'r2')
    print("R2 cross validation score: %0.2f (+/- %0.2f)" % (cvScore.mean(), cvScore.std() * 2))
    
    for i in y_pred:
        y.append(i)

In [ ]:
svr = SVR(kernel='rbf', gamma=1e-4, C=100, epsilon = .1)

y_svr = []

scores(y_svr, svr)

In [ ]:
rf = RandomForestRegressor(random_state = 9, n_estimators = 100, criterion = 'mse')

y_rf = []

scores(y_rf, rf)

In [ ]:
knn = neighbors.KNeighborsRegressor(n_neighbors = 7, weights = 'uniform')

y_knn = []

scores(y_knn, knn)

In [ ]:
dnn = MLPRegressor(
    solver='lbfgs',
    hidden_layer_sizes=100,
    max_iter=10000,
    random_state=987654321,
    activation='identity',
    learning_rate ='invscaling')

y_dnn = []

scores(y_dnn, dnn)

# Prediction

In [ ]:
dfCurrentNames = dfCurrent.iloc[:, 1]
dfCurrentPredict = dfCurrent[['Team Wins', 'Overall Seed', 'PTS', 'TRB', 'AST', 'FG%', 'VORP', 'WS']]

dfCurrent.head()

## SVR 

In [ ]:
svrPredict = svr.predict(dfCurrentPredict)
svrPredict = svrPredict.tolist()

for (i, j) in zip(dfCurrentNames, svrPredict):
    print(i, j)

In [ ]:
svrListUnsorted = [[i, j] for i, j in zip(dfCurrentNames, svrPredict)]
svrDataUnsorted = [row[1] for row in svrListUnsorted]
svrList = sorted(svrListUnsorted, key = itemgetter(1), reverse = True)

svrData = [row[1] for row in svrList]
svrNames = [row[0] for row in svrList]
print(svrList)

x_svr = np.arange(len(svrData))

In [ ]:
plt.style.use('fivethirtyeight')

svr, ax = plt.subplots()

ax.bar(x_svr, svrData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = svrNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 6:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <=6:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    
svr.suptitle("SVM predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

svr.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

svr.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

svr.savefig('svr.png', dpi = 400, bbox_inches = 'tight')

## RF

In [ ]:
rfPredict = rf.predict(dfCurrentPredict)
rfPredict = rfPredict.tolist()

for (i, j) in zip(dfCurrentNames, rfPredict):
    print(i, j)

In [ ]:
rfListUnsorted = [[i, j] for i, j in zip(dfCurrentNames, rfPredict)]
rfDataUnsorted = [row[1] for row in rfListUnsorted]
rfList = sorted(rfListUnsorted, key = itemgetter(1), reverse = True)

rfData = [row[1] for row in rfList]
rfNames = [row[0] for row in rfList]
print(rfList)

x_rf = np.arange(len(rfData))

In [ ]:
plt.style.use('fivethirtyeight')

rf, ax = plt.subplots()

ax.bar(x_rf, rfData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = rfNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 6:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <= 6:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
        
rf.suptitle("RF predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

rf.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

rf.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

rf.savefig('rf.png', dpi = 400, bbox_inches = 'tight')

## k-NN

In [ ]:
knnPredict = knn.predict(dfCurrentPredict)
knnPredict = knnPredict.tolist()

for (i, j) in zip(dfCurrentNames, knnPredict):
    print(i, j)

In [ ]:
knnListUnsorted = [[i, j] for i, j in zip(dfCurrentNames, knnPredict)]
knnDataUnsorted = [row[1] for row in knnListUnsorted]
knnList = sorted(knnListUnsorted, key = itemgetter(1), reverse = True)

knnData = [row[1] for row in knnList]
knnNames = [row[0] for row in knnList]
print(knnList)

x_knn = np.arange(len(knnData))

In [ ]:
plt.style.use('fivethirtyeight')

knn, ax = plt.subplots()

ax.bar(x_knn, knnData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = knnNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 5:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <= 5:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
        
knn.suptitle("KNN predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

knn.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

knn.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

knn.savefig('knn.png', dpi = 400, bbox_inches = 'tight')

## DNN

In [ ]:
dnnPredict = dnn.predict(dfCurrentPredict)
dnnPredict = dnnPredict.tolist()

for (i, j) in zip(dfCurrentNames, dnnPredict):
    print(i, j)

In [ ]:
dnnListUnsorted = [[i, j] for i, j in zip(dfCurrentNames, dnnPredict)]
dnnDataUnsorted = [row[1] for row in dnnListUnsorted]
dnnList = sorted(dnnListUnsorted, key = itemgetter(1), reverse = True)

dnnData = [row[1] for row in dnnList]
dnnNames = [row[0] for row in dnnList]
print(dnnList)

x_dnn = np.arange(len(dnnData))

In [ ]:
plt.style.use('fivethirtyeight')

dnn, ax = plt.subplots()

ax.bar(x_dnn, dnnData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = dnnNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 8:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .05, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() > 7:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <= 7:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
        
dnn.suptitle("DNN predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

dnn.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

dnn.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

dnn.savefig('dnn.png', dpi = 400, bbox_inches = 'tight')

## Average

In [ ]:
avgPredict = []

for (i, j, h, k) in zip(svrDataUnsorted, rfDataUnsorted, knnDataUnsorted, dnnDataUnsorted):
    avgPredict.append((i + j + h + k) / 4)

avgList = [[i, j] for i, j in zip(dfCurrentNames, avgPredict)]
avgList = sorted(avgList, key = itemgetter(1), reverse = True)

avgData = [row[1] for row in avgList]
avgNames = [row[0] for row in avgList]
print(avgList)

x_avg = np.arange(len(avgData))

In [ ]:
plt.style.use('fivethirtyeight')

avg, ax = plt.subplots()

ax.bar(x_avg, avgData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = avgNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 6:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <= 6:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
        
avg.suptitle("Average predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

avg.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

avg.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

avg.savefig('avg.png', dpi = 400, bbox_inches = 'tight')

# Change in predictions

In [ ]:
plt.style.use('fivethirtyeight')
change, ax = plt.subplots()

y_pos = np.arange(len(dfChange['Difference']))

colorList = []

for i in dfChange['Difference'].sort_values(ascending = True):
    if i < 0:
        colorList.append('lightcoral')
    elif i > 0:
        colorList.append('lightgreen')

ax.barh(y_pos, dfChange['Difference'].sort_values(ascending = True), color = colorList, edgecolor = 'white', linewidth = 3)

labels = [i for i in dfChange['Player']]
labels = reversed(labels)

rects = ax.patches
for rect, label in zip(rects, labels):
    height = rect.get_y()
    if rect.get_y() < 2:
        ax.text(rect.get_x() + .005, height, label,
        ha='left', va='bottom', color = 'black')
    elif rect.get_y() >= 2:
        ax.text(rect.get_x() - .005, height, label,
        ha='right', va='bottom', color = 'black')

change.suptitle("Change in predicted MVP share", weight = 'bold', size = 18)
ax.yaxis.set_visible(False)
ax.set_xlabel("ASB average vote share - midseason vote share", size = 14)

change.text(x = -0.02, y = -0.07,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

change.text(x = -0.02, y = -.14,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')
 
change.savefig('change.png', dpi = 400, bbox_inches = 'tight')